## Hope you find this notebook useful :)

In [ ]:
from pandas import read_csv
df = read_csv('/kaggle/input/tabular-playground-series-jan-2021/train.csv')
df

In [ ]:
import lightgbm as lgb
import pandas as pd
import sklearn.metrics
from sklearn.model_selection import train_test_split
import optuna

In [ ]:
def objective(trial):
    #
    df = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/train.csv')
    df = df.drop('id', axis=1)
    target = df['target']
    df = df.drop('target', axis=1)
    #
    train_x, test_x, train_y, test_y = train_test_split(df, target, test_size=0.25)
    train_df = lgb.Dataset(train_x, label=train_y)

    params = {
        "objective": "regression",
        "boosting_type": "gbdt",
        "verbosity": -1,
        "learning_rate": trial.suggest_float("learning_rate", 0.025, 0.075),
        "num_leaves": trial.suggest_int("num_leaves", 160, 224),
        "seed": 16,
        # learning parameters
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 0.001, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", .002, .2, log=True),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.84, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.84, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 5, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 50, 75),
        # IO Parameters
    }
    gbm = lgb.train(params, train_df)
    #
    preds = gbm.predict(train_x)
    train_rmse = sklearn.metrics.mean_squared_error(train_y, preds, squared=False)
    #
    preds = gbm.predict(test_x)
    test_rmse = sklearn.metrics.mean_squared_error(test_y, preds, squared=False)
    #
    # Handle pruning based on the intermediate value.
    # trial.report(test_rmse, epoch)
    # if trial.should_prune():
      #  raise opt.exceptions.TrialPruned()
    #
    return test_rmse

In [ ]:
# Train
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

In [ ]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
trial

In [ ]:
df = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/train.csv')
df = df.drop('id', axis=1)
target = df['target']
df = df.drop('target', axis=1)

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(df, target, test_size=0.25)
train_df = lgb.Dataset(train_x, label=train_y)

In [ ]:
gbm = lgb.train(params=trial.params, train_set=train_df)

In [ ]:
subm = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/test.csv')
subm

In [ ]:
subm['target'] = gbm.predict(subm.drop('id', axis=1))
subm

In [ ]:
subm = subm[['id', 'target']]
subm

In [ ]:
subm.to_csv("submission_lgbm_optuna.csv", index=False)

In [ ]:
# import the modules we'll need
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload, title=title, filename=filename)
    return HTML(html)

# create a random sample dataframe
#df = pd.DataFrame(np.random.randn(50, 4), columns=list('ABCD'))

# create a link to download the dataframe
#

In [ ]:
create_download_link(subm)